In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, tensorflow as tf
import sys, time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, GlobalAveragePooling1D, Conv1DTranspose, LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

pd.set_option('display.precision', 4)

2024-04-09 15:40:56.681061: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 15:40:56.681205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 15:40:56.840447: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 15:40:57.272159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 15:41:02.695495: W tensorflow/compiler/tf2

In [2]:
from tensorflow.keras import backend as K

def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - ss_res/(ss_tot + K.epsilon()))

In [3]:
base_data_train, base_data_test = np.load('../../data/training_data/training_data_1month.npy', allow_pickle=True)

tf.random.set_seed(42)

<h2> Scale Data </h2>

In [4]:
from sklearn.preprocessing import MinMaxScaler

base_data_train, base_data_test = np.load('../../data/training_data/training_data_1month.npy', allow_pickle=True)
scalers = {var_name: MinMaxScaler(feature_range=(-1,1)) for var_name in ['G.air.T', 'G.E_th_I', 'G.sky.T', 'G.E_el_I']}

air_var, sky_var, el_var, th_var = base_data_train[:,:,0], base_data_train[:,:,1], base_data_train[:,:,2], base_data_train[:,:,3]
air_var_test, sky_var_test, el_var_test, th_var_test = base_data_test[:,:,0], base_data_test[:,:,1], base_data_test[:,:,2], base_data_test[:,:,3]

scaled_data_train = np.stack((scalers['G.air.T'].fit_transform(air_var),
                             scalers['G.sky.T'].fit_transform(sky_var),
                             scalers['G.E_el_I'].fit_transform(el_var),
                             scalers['G.E_th_I'].fit_transform(th_var)), axis=-1)
scaled_data_test = np.stack((scalers['G.air.T'].fit_transform(air_var),
                             scalers['G.sky.T'].fit_transform(sky_var),
                             scalers['G.E_el_I'].fit_transform(el_var),
                             scalers['G.E_th_I'].fit_transform(th_var)), axis=-1)
print(scaled_data_train.shape, scaled_data_test.shape)

(108, 730, 4) (108, 730, 4)


In [5]:
def split_base_data(base_data_train, base_data_test, th_indices=[0, 3], el_indices=[1, 2]):
    th_base_data_train, th_base_data_test = base_data_train[:, :, th_indices], base_data_test[:, :, th_indices]
    el_base_data_train, el_base_data_test = base_data_train[:, :, el_indices], base_data_test[:, :, el_indices]
    return th_base_data_train, th_base_data_test, el_base_data_train, el_base_data_test

th_base_data_train, th_base_data_test, el_base_data_train, el_base_data_test = split_base_data(base_data_train, base_data_test)
print(th_base_data_train.shape, th_base_data_test.shape, el_base_data_train.shape, el_base_data_test.shape)

(108, 730, 2) (12, 730, 2) (108, 730, 2) (12, 730, 2)


<h2> CNN Models</h2>

In [6]:
def create_cnn(input_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=24, activation='relu', input_shape=(input_shape)),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Conv1D(filters=64, kernel_size=24, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.25),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(730, activation='linear')
    ])
    return model

In [7]:
def test_cnn(training_data, th_or_el):
    X_train = training_data[:,:,0].reshape(-1,730,1)
    y_train = training_data[:,:,1]

    if th_or_el == 0:
        X_test = scaled_data_test[:,:,0].reshape(-1, 730, 1)  
        y_test = scaled_data_test[:,:,3]
    else:
        X_test = scaled_data_test[:,:,1].reshape(-1, 730, 1)  
        y_test = scaled_data_test[:,:,2]

    X_train, X_train_val, y_train, y_train_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
                                                                  
    model = create_cnn((X_train.shape[1], X_train.shape[2]))
    model.compile(optimizer=Adam(), loss='mse', metrics=['mse', 'mae'])

    early_stopping = EarlyStopping(monitor='mse', patience=10, verbose=1, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='mse', factor=0.5, patience=5, verbose=1)
    model.fit(X_train, y_train, epochs=100, batch_size=16, callbacks=[early_stopping, reduce_lr], verbose=0, validation_data=(X_train_val, y_train_val))

    loss, mse, mae = model.evaluate(X_test, y_test)
    r2 = r_squared(tf.convert_to_tensor(y_test, dtype=tf.float32), tf.convert_to_tensor(model.predict(X_test), dtype=tf.float32))
    
    return {'mse':mse, 'mae':mae, 'r2':r2.numpy()}


In [8]:
num_reps = 100

def test_cnn_wrapper(data, th_or_el=0):
    mse, mae, r2 = [], [], []
    
    # Run each CNN training 10 times to ensure results are significant and not outliers
    for i in range(num_reps):
        print(f'RUN: {i}')
        results = test_cnn(np.random.permutation(data), th_or_el) # permuting the data for each run just to ensure full shuffling
        mse.append(results['mse'])
        mae.append(results['mae'])
        r2.append(results['r2'])

    return {'mse':mse, 'mae':mae, 'r2':r2}
        

<h2> Load data </h2>

In [9]:
def load_data(base_path, context, dataset_names):
    datasets = {}
    for name in dataset_names:
        file_path = f'{base_path}/{context}_{name}_generated_samples.npy'
        datasets[name] = np.load(file_path, allow_pickle=True)
    return datasets

base_path_vae = '../../data/vae_synthetic_data/'
base_path_gan = '../../data/gan_synthetic_data/'
th_context = 'th_v_air'
el_context = 'el_v_sky'

th_vae_datasets = ['b20l5', 'b4l10', 'b16l10', 'b8l20', 'b4l5', 'b16l5', 'b20l3', 'b24l15', 'b8l15', 'b24l50']
th_gan_datasets = ['b32e500', 'b8e1000', 'b20e100', 'b32e100', 'b6e100', 'b8e500']

th_vae_data = load_data(base_path_vae, th_context, th_vae_datasets)
th_gan_data = load_data(base_path_gan, th_context, th_gan_datasets)

el_vae_datasets = ['b4l3', 'b8l10', 'b20l3', 'b20l20', 'b24l3', 'b32l3', 'b32l5']
el_gan_datasets = ['b4e1000','b6e100','b8e500','b8e1000', 'b10e100', 'b12e100', 'b12e500', 'b16e500', 'b20e500', 'b24e1000', 'b32e1000']

el_vae_data = load_data(base_path_vae, el_context, el_vae_datasets)
el_gan_data = load_data(base_path_gan, el_context, el_gan_datasets)

In [10]:
print(len(th_vae_data), [th_vae_data[th_vae_datasets[i]].shape for i in range(len(th_vae_data))])
print(len(th_gan_data), [th_gan_data[th_gan_datasets[i]].shape for i in range(len(th_gan_data))])

print(len(el_vae_data), [el_vae_data[el_vae_datasets[i]].shape for i in range(len(el_vae_data))])
print(len(el_gan_data), [el_gan_data[el_gan_datasets[i]].shape for i in range(len(el_gan_data))])

10 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
6 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
7 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]
11 [(1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2), (1000, 730, 2)]


<h2> Train CNNs </h2>

<h3> Ground Truth Datasets </h3>

In [24]:
pd.DataFrame(test_cnn_wrapper(scaled_data_train[:,:,[0,3]], 0)).to_csv('../../data/evaluation_results/th_gt.csv', index=False)
pd.DataFrame(test_cnn_wrapper(scaled_data_train[:,:,[1,2]], 1)).to_csv('../../data/evaluation_results/el_gt.csv', index=False)

RUN: 0
4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 76: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 59: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5
4/4 [==============================] - 0s 2ms/step
RUN: 6
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 99: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8
4/4 [==============================] - 0s 2ms/step
RUN: 9
4/4 [==============================] - 0s 2ms/step
RUN: 10
4/4 [==============================] - 0s 2ms/step
RUN: 11

Epoch 97: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s

4/4 [==============================] - 0s 2ms/step
RUN: 91
4/4 [==============================] - 0s 2ms/step
RUN: 92
4/4 [==============================] - 0s 2ms/step
RUN: 93

Epoch 86: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 94
4/4 [==============================] - 0s 2ms/step
RUN: 95
4/4 [==============================] - 0s 2ms/step
RUN: 96

Epoch 72: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 97
4/4 [==============================] - 0s 2ms/step
RUN: 98
4/4 [==============================] - 0s 2ms/step
RUN: 99

Epoch 59: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 0
4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2
4/4 [==============================] - 0s 2ms/step
RUN: 3
4/4

4/4 [==============================] - 0s 2ms/step
RUN: 96
4/4 [==============================] - 0s 2ms/step
RUN: 97
4/4 [==============================] - 0s 2ms/step
RUN: 98
4/4 [==============================] - 0s 2ms/step
RUN: 99
4/4 [==============================] - 0s 2ms/step


In [22]:
th_gt, el_gt = pd.read_csv('../../data/evaluation_results/th_gt.csv'), pd.read_csv('../../data/evaluation_results/el_gt.csv')

print(np.mean(th_gt['mse']), np.mean(el_gt['mse']))

0.09846784219145771 0.286872461438179


<h3> Synthetic Datasets </h3>

In [11]:
def evaluate_and_save_cnn(data_dicts, model_type, column_index, base_path='../../data/evaluation_results/'):
    for dataset_name, dataset in data_dicts.items():
        cnn_test_results = test_cnn_wrapper(dataset[0:216, :, :], column_index)

        result_df = pd.DataFrame(cnn_test_results)
        result_file_name = f'{base_path}{model_type}_{"th" if column_index==0 else "el"}_{dataset_name}.csv'
        result_df.to_csv(result_file_name)
        print(f'CNN results for {dataset_name}: {result_df}')
        print(f'Saved CNN evaluation results for {dataset_name} to: {result_file_name}')


In [ ]:
evaluate_and_save_cnn(th_vae_data, 'vae', 0)
evaluate_and_save_cnn(th_gan_data, 'gan', 0)


In [ ]:
evaluate_and_save_cnn(el_vae_data, 'vae', 1)
evaluate_and_save_cnn(el_gan_data, 'gan', 1)


<h3> Blended Datasets </h3>

In [12]:
def evaluate_and_save_cnn_blended(data_dicts, model_type, column_index, base_path='../../data/evaluation_results/'):
    for dataset_name, dataset in data_dicts.items():
        scaled_data = scaled_data_train[:,:,[0,3]] if column_index==0 else scaled_data_train[:,:,[1,2]]
        cnn_test_results = test_cnn_wrapper(np.concatenate((scaled_data, dataset[0:216,:,:]), axis=0), column_index)

        result_df = pd.DataFrame(cnn_test_results)
        result_file_name = f'{base_path}{model_type}_blended_{"th" if column_index==0 else "el"}_{dataset_name}.csv'
        result_df.to_csv(result_file_name)
        print(f'CNN results for {dataset_name}: {result_df}')
        print(f'Saved CNN evaluation results for {dataset_name} to: {result_file_name}')


In [ ]:
evaluate_and_save_cnn_blended(th_vae_data, 'vae', 0)
evaluate_and_save_cnn_blended(th_gan_data, 'gan', 0)


In [97]:
evaluate_and_save_cnn_blended(el_vae_data, 'vae', 1)
evaluate_and_save_cnn_blended(el_gan_data, 'gan', 1)


RUN: 0
4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2

Epoch 97: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 3
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 85: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 95: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 6

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 90: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8
4/4 [==============================] - 0s 2ms/step
RUN: 9
4/4

4/4 [==============================] - 0s 1ms/step
RUN: 1

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 2

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 98: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 84: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 88: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 6

Epoch 80: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 75: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/st

4/4 [==============================] - 0s 2ms/step
RUN: 1
4/4 [==============================] - 0s 2ms/step
RUN: 2
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 70: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 6
4/4 [==============================] - 0s 2ms/step
RUN: 7

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 8

Epoch 91: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 9
4/4 [==============================] - 0s 2ms/step
CNN results for b16e500:         mse 

In [ ]:
evaluate_and_save_cnn(th_vae_data, 'vae', 0)
evaluate_and_save_cnn(th_gan_data, 'gan', 0)

evaluate_and_save_cnn_blended(th_vae_data, 'vae', 0)
evaluate_and_save_cnn_blended(th_gan_data, 'gan', 0)

evaluate_and_save_cnn(el_vae_data, 'vae', 1)
evaluate_and_save_cnn(el_gan_data, 'gan', 1)

evaluate_and_save_cnn_blended(el_vae_data, 'vae', 1)
evaluate_and_save_cnn_blended(el_gan_data, 'gan', 1)

RUN: 0


2024-04-09 15:43:18.429048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6876 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1
2024-04-09 15:43:26.891783: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-09 15:43:37.315788: I external/local_xla/xla/service/service.cc:168] XLA service 0x2baff0e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-09 15:43:37.315831: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-04-09 15:43:37.421120: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712673817.784725    4520 device_compiler.h:186] Compiled cluster using XLA!  This


Epoch 70: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 99: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 1

Epoch 42: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 93: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 2

Epoch 56: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 3

Epoch 92: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 4

Epoch 39: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 5

Epoch 62: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 94: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [===


Epoch 100: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
4/4 [==============================] - 0s 2ms/step
RUN: 24

Epoch 37: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 25

Epoch 59: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 71: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 82: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 93: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
4/4 [==============================] - 0s 2ms/step
RUN: 26

Epoch 66: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 2ms/step
RUN: 27

Epoch 32: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 81: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [==============================] - 0s 2ms/step
RUN: 28

Epo


Epoch 34: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 59: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 85: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 99: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
4/4 [==============================] - 0s 2ms/step
RUN: 48

Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 45: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 66: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 92: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 99: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
4/4 [==============================] - 0s 2ms/step
RUN: 49

Epoch 78: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 89: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
4/4 [=====================

<h2> Select and compare data </h2>

In [10]:
import glob

def find_best_model(prefix):
    folder_path = '../../data/evaluation_results/cnn/'

    file_paths = glob.glob(f'{folder_path}{prefix}*.csv')
    
    mse_vals = {}
    best_val, best_df, best_name = float('inf'), None, ''
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        mse_val = df['mse'].iloc[0]
        mse_vals[file_path] = mse_val
        
        if mse_val < best_val:
            best_val, best_df = mse_val, df
            best_name = file_path.split('/')[-1]

    sorted_mse_vals = sorted(mse_vals.items(), key=lambda item: item[1])
    
    for name, value in sorted_mse_vals:
        print(name.split('/')[-1], value)
        
    print(f'\nDataFrame with lowest MSE ({best_name}):\n{best_df}\n')

In [18]:
print('Context: Thermal')
find_best_model('vae_th')
find_best_model('gan_th')

print('\nContext: Electrical')
find_best_model('vae_el')
find_best_model('gan_el')


Context: Thermal
cnn\vae_th_b20l3.csv 0.1190055929124355
cnn\vae_th_b8l15.csv 0.1265775240957737
cnn\vae_th_b16l5.csv 0.1314404964447021
cnn\vae_th_b4l5.csv 0.1364974901080131
cnn\vae_th_b20l5.csv 0.1520956456661224
cnn\vae_th_b24l15.csv 0.1523252636194229
cnn\vae_th_b4l10.csv 0.1532680049538612
cnn\vae_th_b24l50.csv 0.1778552532196045
cnn\vae_th_b16l10.csv 0.1802604213356971
cnn\vae_th_b8l20.csv 0.5562011629343033

DataFrame with lowest MSE (cnn\vae_th_b20l3.csv):
   Unnamed: 0    mse     mae      r2
0           0  0.119  0.2626  0.3321

cnn\gan_th_b8e1000.csv 0.2068981036543846
cnn\gan_th_b32e100.csv 0.2844437450170517
cnn\gan_th_b8e500.csv 0.2867301911115646
cnn\gan_th_b20e100.csv 0.3444884002208709
cnn\gan_th_b6e100.csv 0.4662655055522918
cnn\gan_th_b32e500.csv 0.5470245212316514

DataFrame with lowest MSE (cnn\gan_th_b8e1000.csv):
   Unnamed: 0     mse     mae      r2
0           0  0.2069  0.3807 -0.1612


Context: Electrical
cnn\vae_el_b8l10.csv 0.4488880276679993
cnn\vae_el_b24

In [19]:
print('Context: Thermal')
find_best_model('vae_blended_th')
find_best_model('gan_blended_th')

print('\nContext: Electrical')
find_best_model('vae_blended_el')
find_best_model('gan_blended_el')

Context: Thermal
cnn\vae_blended_th_b20l3.csv 0.0781151488423347
cnn\vae_blended_th_b4l10.csv 0.0924433469772338
cnn\vae_blended_th_b8l15.csv 0.0932417020201683
cnn\vae_blended_th_b4l5.csv 0.0968162976205349
cnn\vae_blended_th_b24l15.csv 0.09766411408782
cnn\vae_blended_th_b16l5.csv 0.1016411677002906
cnn\vae_blended_th_b20l5.csv 0.1065429620444774
cnn\vae_blended_th_b24l50.csv 0.1090219363570213
cnn\vae_blended_th_b16l10.csv 0.120886443555355
cnn\vae_blended_th_b8l20.csv 0.1320243626832962

DataFrame with lowest MSE (cnn\vae_blended_th_b20l3.csv):
   Unnamed: 0     mse     mae      r2
0           0  0.0781  0.2037  0.5616

cnn\gan_blended_th_b20e100.csv 0.0895717576146125
cnn\gan_blended_th_b32e500.csv 0.1010843776166439
cnn\gan_blended_th_b8e1000.csv 0.1029256179928779
cnn\gan_blended_th_b32e100.csv 0.1281974203884601

DataFrame with lowest MSE (cnn\gan_blended_th_b20e100.csv):
   Unnamed: 0     mse     mae      r2
0           0  0.0896  0.2154  0.4973


Context: Electrical
cnn\vae_b